# ***CS 230 DEEP LEARNING FASHION OUTFIT LOOKBOOK [CODE]***

***Author:*** Aishwarya Rameshkumar


***Overview:*** 
The fashion and design industry is a globally-encompassing \$2.4 trillion dollar industry which has been around for centuries; however, throughout this time, the traditional approach to design with human designers has largely stayed the same. With recent advancements, we now have the opportunity to utilize strides in deep learning to rethink the approach to fashion design and generation. This project explore the use of DCGANs as well as other deep learning techniques such as ANN Classifiers in the exploration and generation of new fashion outfit looks. The final deliverable is an end-to-end pipeline including GAN/Classifier models, analysis modules, and more which encompasses the entire machine learning cycle and results in generated clothing images near state-of-the-art image quality and the generation of a new model-based fashion outfit for a lookbook.

## **I. PRELIMINARY SETUP AND IMPORTS**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
cd gdrive/My Drive/CS230/

In [0]:
from tensorflow import keras
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import os
from glob import glob
import datetime
import time
import random
import copy
import re
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

print(tf.__version__)

## **II. DCGAN RELATED FUNCTIONS**

In [0]:
def generator(z, output_channel_dim, training):
    with tf.variable_scope("generator", reuse= not training):
        
        # 8x8x1024
        fully_connected = tf.layers.dense(z, 8*8*1024)
        fully_connected = tf.reshape(fully_connected, (-1, 8, 8, 1024))
        fully_connected = tf.nn.leaky_relu(fully_connected)

        # 8x8x1024 -> 16x16x512
        trans_conv1 = tf.layers.conv2d_transpose(inputs=fully_connected,
                                                 filters=512,
                                                 kernel_size=[5,5],
                                                 strides=[2,2],
                                                 padding="SAME",
                                                 kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV),
                                                 name="trans_conv1")
        batch_trans_conv1 = tf.layers.batch_normalization(inputs = trans_conv1,
                                                          training=training,
                                                          epsilon=EPSILON,
                                                          name="batch_trans_conv1")
        trans_conv1_out = tf.nn.leaky_relu(batch_trans_conv1,
                                           name="trans_conv1_out")
        
        # 16x16x512 -> 32x32x256
        trans_conv2 = tf.layers.conv2d_transpose(inputs=trans_conv1_out,
                                                 filters=256,
                                                 kernel_size=[5,5],
                                                 strides=[2,2],
                                                 padding="SAME",
                                                 kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV),
                                                 name="trans_conv2")
        batch_trans_conv2 = tf.layers.batch_normalization(inputs = trans_conv2,
                                                          training=training,
                                                          epsilon=EPSILON,
                                                          name="batch_trans_conv2")
        trans_conv2_out = tf.nn.leaky_relu(batch_trans_conv2,
                                           name="trans_conv2_out")
        
        # 32x32x256 -> 64x64x128
        trans_conv3 = tf.layers.conv2d_transpose(inputs=trans_conv2_out,
                                                 filters=128,
                                                 kernel_size=[5,5],
                                                 strides=[2,2],
                                                 padding="SAME",
                                                 kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV),
                                                 name="trans_conv3")
        batch_trans_conv3 = tf.layers.batch_normalization(inputs = trans_conv3,
                                                          training=training,
                                                          epsilon=EPSILON,
                                                          name="batch_trans_conv3")
        trans_conv3_out = tf.nn.leaky_relu(batch_trans_conv3,
                                           name="trans_conv3_out")
        
        # 64x64x128 -> 128x128x64
        trans_conv4 = tf.layers.conv2d_transpose(inputs=trans_conv3_out,
                                                 filters=64,
                                                 kernel_size=[5,5],
                                                 strides=[2,2],
                                                 padding="SAME",
                                                 kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV),
                                                 name="trans_conv4")
        batch_trans_conv4 = tf.layers.batch_normalization(inputs = trans_conv4,
                                                          training=training,
                                                          epsilon=EPSILON,
                                                          name="batch_trans_conv4")
        trans_conv4_out = tf.nn.leaky_relu(batch_trans_conv4,
                                           name="trans_conv4_out")
        
        # 128x128x64 -> 128x128x3
        logits = tf.layers.conv2d_transpose(inputs=trans_conv4_out,
                                            filters=3,
                                            kernel_size=[5,5],
                                            strides=[1,1],
                                            padding="SAME",
                                            kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV),
                                            name="logits")
        out = tf.tanh(logits, name="out")
        return out

def discriminator(x, reuse):
    with tf.variable_scope("discriminator", reuse=reuse): 
        
        # 128*128*3 -> 64x64x64 
        conv1 = tf.layers.conv2d(inputs=x,
                                 filters=64,
                                 kernel_size=[5,5],
                                 strides=[2,2],
                                 padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV),
                                 name='conv1')
        batch_norm1 = tf.layers.batch_normalization(conv1,
                                                    training=True,
                                                    epsilon=EPSILON,
                                                    name='batch_norm1')
        conv1_out = tf.nn.leaky_relu(batch_norm1,
                                     name="conv1_out")
        
        # 64x64x64-> 32x32x128 
        conv2 = tf.layers.conv2d(inputs=conv1_out,
                                 filters=128,
                                 kernel_size=[5, 5],
                                 strides=[2, 2],
                                 padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV),
                                 name='conv2')
        batch_norm2 = tf.layers.batch_normalization(conv2,
                                                    training=True,
                                                    epsilon=EPSILON,
                                                    name='batch_norm2')
        conv2_out = tf.nn.leaky_relu(batch_norm2,
                                     name="conv2_out")
        
        # 32x32x128 -> 16x16x256  
        conv3 = tf.layers.conv2d(inputs=conv2_out,
                                 filters=256,
                                 kernel_size=[5, 5],
                                 strides=[2, 2],
                                 padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV),
                                 name='conv3')
        batch_norm3 = tf.layers.batch_normalization(conv3,
                                                    training=True,
                                                    epsilon=EPSILON,
                                                    name='batch_norm3')
        conv3_out = tf.nn.leaky_relu(batch_norm3,
                                     name="conv3_out")
        
        # 16x16x256 -> 16x16x512
        conv4 = tf.layers.conv2d(inputs=conv3_out,
                                 filters=512,
                                 kernel_size=[5, 5],
                                 strides=[1, 1],
                                 padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV),
                                 name='conv4')
        batch_norm4 = tf.layers.batch_normalization(conv4,
                                                    training=True,
                                                    epsilon=EPSILON,
                                                    name='batch_norm4')
        conv4_out = tf.nn.leaky_relu(batch_norm4,
                                     name="conv4_out")
        
        # 16x16x512 -> 8x8x1024
        conv5 = tf.layers.conv2d(inputs=conv4_out,
                                filters=1024,
                                kernel_size=[5, 5],
                                strides=[2, 2],
                                padding="SAME",
                                kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV),
                                name='conv5')
        batch_norm5 = tf.layers.batch_normalization(conv5,
                                                    training=True,
                                                    epsilon=EPSILON,
                                                    name='batch_norm5')
        conv5_out = tf.nn.leaky_relu(batch_norm5,
                                     name="conv5_out")

        flatten = tf.reshape(conv5_out, (-1, 8*8*1024))
        logits = tf.layers.dense(inputs=flatten,
                                 units=1,
                                 activation=None)
        out = tf.sigmoid(logits)
        return out, logits

In [0]:
def model_loss(input_real, input_z, output_channel_dim):
    g_model = generator(input_z, output_channel_dim, True)

    noisy_input_real = input_real + tf.random_normal(shape=tf.shape(input_real),
                                                     mean=0.0,
                                                     stddev=random.uniform(0.0, 0.1),
                                                     dtype=tf.float32)
    
    d_model_real, d_logits_real = discriminator(noisy_input_real, reuse=False)
    d_model_fake, d_logits_fake = discriminator(g_model, reuse=True)
    
    d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real,
                                                                         labels=tf.ones_like(d_model_real)*random.uniform(0.9, 1.0)))
    d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake,
                                                                         labels=tf.zeros_like(d_model_fake)))
    d_loss = tf.reduce_mean(0.5 * (d_loss_real + d_loss_fake))
    g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake,
                                                                    labels=tf.ones_like(d_model_fake)))
    return d_loss, g_loss

def model_optimizers(d_loss, g_loss):
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith("generator")]
    d_vars = [var for var in t_vars if var.name.startswith("discriminator")]
    
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    gen_updates = [op for op in update_ops if op.name.startswith('generator')]
    
    with tf.control_dependencies(gen_updates):
        d_train_opt = tf.train.AdamOptimizer(learning_rate=LR_D, beta1=BETA1).minimize(d_loss, var_list=d_vars)
        g_train_opt = tf.train.AdamOptimizer(learning_rate=LR_G, beta1=BETA1).minimize(g_loss, var_list=g_vars)  
    return d_train_opt, g_train_opt

def model_inputs(real_dim, z_dim):
    inputs_real = tf.placeholder(tf.float32, (None, *real_dim), name='inputs_real')
    inputs_z = tf.placeholder(tf.float32, (None, z_dim), name="input_z")
    learning_rate_G = tf.placeholder(tf.float32, name="lr_g")
    learning_rate_D = tf.placeholder(tf.float32, name="lr_d")
    return inputs_real, inputs_z, learning_rate_G, learning_rate_D

In [0]:
def test(sess, input_z, out_channel_dim, epoch):
    example_z = np.random.uniform(-1, 1, size=[SAMPLES_TO_SHOW, input_z.get_shape().as_list()[-1]])
    samples = sess.run(generator(input_z, out_channel_dim, False), feed_dict={input_z: example_z})
    sample_images = [((sample + 1.0) * 127.5).astype(np.uint8) for sample in samples]
    show_samples(sample_images, OUTPUT_DIR_GAN + "samples", epoch)

def summarize_epoch(epoch, duration, sess, d_losses, g_losses, input_z, data_shape):
    minibatch_size = int(data_shape[0]//BATCH_SIZE)
    print("Epoch {}/{}".format(epoch, EPOCHS),
          "\nDuration: {:.5f}".format(duration),
          "\nD Loss: {:.5f}".format(np.mean(d_losses[-minibatch_size:])),
          "\nG Loss: {:.5f}".format(np.mean(g_losses[-minibatch_size:])))
    fig, ax = plt.subplots()
    plt.plot(d_losses, label='Discriminator', alpha=0.6)
    plt.plot(g_losses, label='Generator', alpha=0.6)
    plt.title("Losses")
    plt.legend()
    plt.savefig(OUTPUT_DIR_GAN + "losses_" + str(epoch) + ".png")
    plt.show()
    plt.close()
    test(sess, input_z, data_shape[3], epoch)

def get_batches(data):
    batches = []
    for i in range(int(data.shape[0]//BATCH_SIZE)):
        batch = data[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
        augmented_images = []
        for img in batch:
            image = Image.fromarray(img)
            if random.choice([True, False]):
                image = image.transpose(Image.FLIP_LEFT_RIGHT)
            augmented_images.append(np.asarray(image))
        batch = np.asarray(augmented_images)
        normalized_batch = (batch / 127.5) - 1.0
        batches.append(normalized_batch)
    return batches

In [0]:
def train(get_batches, data_shape, checkpoint_to_load=None):
    input_images, input_z, lr_G, lr_D = model_inputs(data_shape[1:], NOISE_SIZE)
    d_loss, g_loss = model_loss(input_images, input_z, data_shape[3])
    d_opt, g_opt = model_optimizers(d_loss, g_loss)
    
    saver = tf.train.Saver()
    sess = tf.Session()
    if checkpoint_to_load:
      print("Loading checkpoint model")
      saved_location =  CHECKPOINT_DIR + CHECKPOINT_META_FILE
      saved_model = tf.train.import_meta_graph(saved_location)
      saved_model.restore(sess,tf.train.latest_checkpoint(CHECKPOINT_DIR))
    else:
      sess.run(tf.global_variables_initializer())
    epoch = 0
    iteration = 0
    d_losses = []
    g_losses = []
        
    for epoch in range(EPOCHS):        
        epoch += 1
        start_time = time.time()

        for batch_images in get_batches:
            iteration += 1
            batch_z = np.random.uniform(-1, 1, size=(BATCH_SIZE, NOISE_SIZE))
            _ = sess.run(d_opt, feed_dict={input_images: batch_images, input_z: batch_z, lr_D: LR_D})
            _ = sess.run(g_opt, feed_dict={input_images: batch_images, input_z: batch_z, lr_G: LR_G})
            d_losses.append(d_loss.eval(feed_dict={input_z: batch_z, input_images: batch_images}, session=sess))
            g_losses.append(g_loss.eval(feed_dict={input_z: batch_z}, session=sess))

        summarize_epoch(epoch, time.time()-start_time, sess, d_losses, g_losses, input_z, data_shape)

        if epoch%5 == 0:
          savepath = MODEL_SAVE_DIR + 'model_checkpoint_epoch_' + str(epoch) 
          saver.save(sess, savepath)

In [0]:
def show_samples(sample_images, name, epoch):
    figure, axes = plt.subplots(1, len(sample_images), figsize = (IMAGE_SIZE, IMAGE_SIZE))
    for index, axis in enumerate(axes):
        axis.axis('off')
        image_array = sample_images[index]
        axis.imshow(image_array)
        image = Image.fromarray(image_array)
        image.save(name+"_"+str(epoch)+"_"+str(index)+".png") 
    plt.savefig(name+"_"+str(epoch)+".png", bbox_inches='tight', pad_inches=0)
    plt.show()
    plt.close()

In [0]:
# Referenced code: https://github.com/gsurma/image_generator

## **II. ANN/CLASSIFIER RELATED FUNCTIONS**

In [0]:
def parse_labels(input_dir, annos, names2labels_dict):
  labels_df = pd.DataFrame(columns = ["image id", "category", "category_label"])
  for file in glob(input_dir + '*'):
    image_file = file.split(sep="/")
    image_id = image_file[2].split(sep=".")[0]
    category = annos.loc[annos.id == int(image_id), ['articleType']].values.tolist()[0][0]
    category_label = names2labels_dict[category]
    labels_df = labels_df.append({"image id": image_id, "category": category, "category_label": category_label}, ignore_index=True)
  final_labels = labels_df["category_label"].to_numpy()
  print(labels_df.head())
  return labels_df, final_labels

In [0]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)

  if predicted_label == true_label:
    color = 'green'
    plt.xlabel("{}".format(category_labels2names_dict[predicted_label]),
                                color=color)
  else:
    color = 'red'
    plt.xlabel("{} ({})".format(category_labels2names_dict[predicted_label],
                                category_labels2names_dict[true_label]),
                                color=color)
  

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  thisplot = plt.scatter([i for i in range(len(predictions_array))], predictions_array, color="#777777");
  max_value_point = np.argmax(predictions[i])
  thisplot = plt.scatter([true_label], predictions_array[true_label], color='r');
  thisplot = plt.scatter([max_value_point], predictions_array[max_value_point], color='g');
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

In [0]:
#Referenced code tutorial: https://www.tensorflow.org/tutorials/keras/classification

## **IV. MAIN INPUT INITIALIZATION AND DIRECTORY SETUP**

In [0]:
# Paths to images and directory setup
INPUT_DATA_DIR = "fashion_images_s/images/" #Input images folder
INPUT_DATA_DIR_TEST = "fashion_images_s/images_test/" #Input images test set folder 
INPUT_ANNO_PATH = "fashion_images_s/styles.csv" #Input images csv with annotations of image category, style, etc.

#Output directories to save files for experiment
OUTPUT_DIR = './final_test_output_2/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

OUTPUT_DIR_GAN = OUTPUT_DIR + "gan/"
if not os.path.exists(OUTPUT_DIR_GAN):
    os.makedirs(OUTPUT_DIR_GAN)

OUTPUT_DIR_CNN = OUTPUT_DIR + "cnn/"
if not os.path.exists(OUTPUT_DIR_CNN):
    os.makedirs(OUTPUT_DIR_CNN)

MODEL_SAVE_DIR = OUTPUT_DIR + 'models/'
if not os.path.exists(MODEL_SAVE_DIR):
    os.makedirs(MODEL_SAVE_DIR)

#Point to file to load if input images were already processed and saved
LOAD_SAVED_IMAGES = True
INPUT_IMAGES_SAVED_PATH = 'fashion_images_s/images_input_images_saved.npy'
TEST_IMAGES_SAVED_PATH = 'fashion_images_s/images_test_images_saved.npy'

#Point to checkpoint to resume model training from 
LOAD_CHECKPOINT = False
CHECKPOINT_DIR = './ark_test_output_5_3_models/'
CHECKPOINT_META_FILE = 'model_save_test_epoch_10.meta'

In [0]:
# Hyperparameters Initialization
IMAGE_SIZE = 128
NOISE_SIZE = 95
LR_D = 0.0001
LR_G = 0.0001
LR_CNN = 0.0001
BATCH_SIZE = 64
EPOCHS = 300
BETA1 = 0.3
WEIGHT_INIT_STDDEV = 0.02
EPSILON = 0.000008
SAMPLES_TO_SHOW = 5
EPOCHS_CNN = 300


In [0]:
# Input images processing/loading 
if LOAD_SAVED_IMAGES == True:
  print("Loading processed images...")
  input_images = np.load(INPUT_IMAGES_SAVED_PATH)
  input_images_cnn = copy.deepcopy(input_images)
  test_images = np.load(TEST_IMAGES_SAVED_PATH)
else:
  print("Processing images...")
  input_images = np.asarray([np.asarray(Image.open(file).convert('RGB').resize((IMAGE_SIZE, IMAGE_SIZE))) for file in glob(INPUT_DATA_DIR + '*')])
  input_images_cnn = copy.deepcopy(input_images)
  test_images = np.asarray([np.asarray(Image.open(file).convert('RGB').resize((IMAGE_SIZE, IMAGE_SIZE))) for file in glob(INPUT_DATA_DIR_TEST + '*')])

print ("Input Sample: ")

if not os.path.exists(INPUT_IMAGES_SAVED_PATH):
  np.save(INPUT_IMAGES_SAVED_PATH, input_images)
  print("Processed images array saved.")
if not os.path.exists(TEST_IMAGES_SAVED_PATH):
  np.save(TEST_IMAGES_SAVED_PATH, test_images)
  print("Processed test images array saved.")

np.random.shuffle(input_images)

np.seterr('raise')

sample_images = random.sample(list(input_images), SAMPLES_TO_SHOW)
show_samples(sample_images, OUTPUT_DIR_GAN + "inputs", 0)

## **V. ANN/CLASSIFIER MODEL SETUP/TRAINING/TESTING**

In [0]:
#Parse labels for classifier
annotations = pd.read_csv(INPUT_ANNO_PATH)  
annotations = annotations.drop(columns=['Unnamed: 10', 'Unnamed: 11']).sort_values(by=['id'])

category_names = annotations.articleType.unique().tolist()
category_names2labels_dict = {}
count = 0
for i, c in enumerate(category_names):
  category_names2labels_dict[c] = i
category_labels2names_dict = {value : key for (key, value) in category_names2labels_dict.items()}

In [0]:
annotations

In [0]:
#Plot distribution of article type
p = annotations.articleType.value_counts()
p.plot(kind='bar', title="Article Type vs. Counts")
annotations.articleType.value_counts()

count=0
for i in range(p.shape[0]):
  if p[i] >= 1000:
    count = count + 1


In [0]:
#Generate image labels for train and test sets
train_labels_df, input_images_labels = parse_labels(INPUT_DATA_DIR, annotations, category_names2labels_dict)
test_labels_df, test_images_labels = parse_labels(INPUT_DATA_DIR_TEST, annotations, category_names2labels_dict)
input_images_cnn = input_images_cnn / 255.0

In [0]:
#Plot sample of train images with corresponding label to verify
plt.figure(figsize=(10,10))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(input_images_cnn[i], cmap=plt.cm.binary)
    plt.xlabel(category_labels2names_dict[input_images_labels[i]])
plt.show()

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(128, 128, 3)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(len(category_names))
])

opt = keras.optimizers.Adam(learning_rate=LR_CNN)
model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(input_images_cnn, input_images_labels, epochs=EPOCHS_CNN)

In [0]:
test_loss, test_acc = model.evaluate(test_images, test_images_labels, verbose=2)

print('\nTest accuracy:', test_acc)

probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(test_images)
print(predictions[0])
print(np.argmax(predictions[0]))
print(test_images_labels[0])

In [0]:
# Plot the first X test images, their predicted label, and the next highest prob label
num_rows = 2
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
idx_plotted = []
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  idx = random.choice([x for x in range(len(test_images))])
  while (idx in idx_plotted):
    idx = random.choice([x for x in range(len(test_images))])
  plot_image(idx, predictions[idx], test_images_labels, test_images)
  idx_plotted.append(idx)
plt.tight_layout()
plt.show()

## **VI. DCGAN MODEL SETUP/TRAINING/IMAGE GENERATION**

In [0]:
#TRAIN DCGAN AND GENERATE IMAGES
with tf.Graph().as_default():
    train(get_batches(input_images), input_images.shape, checkpoint_to_load = LOAD_CHECKPOINT)

## **VII. PREDICTION/CLASSIFICATION OF GAN GENERATED IMAGES**

In [0]:
#Process GAN generated images into correct format and predict category labels

INPUT_GAN_IMAGES = "./ark_test_output_5_3/" #OUTPUT_DIR_GAN

print("Processing images...")
input_gan_images = np.asarray([np.asarray(Image.open(file).convert('RGB').resize((IMAGE_SIZE, IMAGE_SIZE))) for file in glob(INPUT_GAN_IMAGES + '*') if re.search("samples_\d+_", file)])

print ("Input: " + str(input_gan_images.shape))

sample_images = random.sample(list(input_gan_images), SAMPLES_TO_SHOW)
show_samples(sample_images, "classification_test_output_1/", 0)

In [0]:
gan_images_predictions = probability_model.predict(input_gan_images)
print(gan_images_predictions)

In [0]:
def plot_image_gan(i, predictions_array, img):
  predictions_array, img = predictions_array, img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  
  color = 'blue'
  plt.xlabel("{}".format(category_labels2names_dict[predicted_label]),
                                color=color)

In [0]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image_gan(i, gan_images_predictions[i], input_gan_images)
plt.tight_layout()
plt.show()

## **VIII. FINAL OUTFIT PAIRING AND LOOKBOOK RECOMMENDATION**

In [0]:
def get_category_img(images, predictions):
  category_img_dict = {}
  for i in range(len(images)):
    predicted_label = np.argmax(predictions[i])
    category_label = category_labels2names_dict[predicted_label]
    if category_label in category_img_dict:
      category_img_dict[category_label].append(i)
    else:
      category_img_dict[category_label] = [i]

  return category_img_dict

gan_category_images = get_category_img(input_gan_images, gan_images_predictions)

In [0]:
pairings = [['Shirts', 'Jeans', 'Sports Shoes', 'Wallets'],
            ['Sweaters', 'Trousers', 'Casual Shoes', 'Watches'],
            ['Kurtas', 'Shorts', 'Heels', 'Belts'],
            ['Dresses', 'Heels', 'Handbags', 'Scarves'],
            ['Tshirts', 'Jeans', 'Jackets', 'Casual Shoes'],
            ['Sweaters', 'Shorts', 'Jackets', 'Sports Shoes', 'Caps'],
            ['Tops', 'Jeans', 'Casual Shoes', 'Backpacks', 'Belts'],
            ['Sweatshirts', 'Shorts', 'Sports Shoes', 'Sunglasses', 'Waist Pouch'],
            ['Shirts', 'Trunk', 'Innerwear Vests', 'Wallets', 'Casual Shoes'],
            ['Tshirts', 'Shorts', 'Heels', 'Handbags'],
            ['Kurtis', 'Heels', 'Handbags']]

In [0]:
def generate_new_look(images, category_images):
  final_pairing = []
  final_pairing_categories = []
  pairing = random.choice(pairings)
  for item in pairing:
    image_ind = random.choice(category_images[item])
    final_pairing.append(images[image_ind])
    final_pairing_categories.append(item)
  return final_pairing, final_pairing_categories

def plot_final_outfit(i, images, images_categories):
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(images[i], cmap=plt.cm.binary)

  color = 'blue'
  plt.xlabel("{}".format(images_categories[i]), color=color)

def show_final_outfit(images, images_categories):
    figure, axes = plt.subplots(1, len(images), figsize = (IMAGE_SIZE, IMAGE_SIZE))
    for index, axis in enumerate(axes):
        axis.axis('off')
        image_array = images[index]
        axis.imshow(image_array)
        axis.set_title("{}".format(images_categories[index]), fontsize=60) #.xlabel("{}".format(images_categories[index]), color=color)
        image = Image.fromarray(image_array)
    plt.show()
    plt.close()


In [0]:
#Generate 2 sample lookbook outfits
print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------FINAL EXAMPLE LOOKBOOK OUTFIT: ---------------------------------------------------------------------------")
print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------\n\n")
final_outfit, final_outfit_cat = generate_new_look(input_gan_images, gan_category_images)
show_final_outfit(final_outfit, final_outfit_cat)
print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------")

In [0]:
print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------FINAL EXAMPLE LOOKBOOK OUTFIT: ---------------------------------------------------------------------------")
print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------\n\n")
final_outfit, final_outfit_cat = generate_new_look(input_gan_images, gan_category_images)
show_final_outfit(final_outfit, final_outfit_cat)
print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------")